<h3>0.0 CARGA DE BIBLIOTECAS, VARIABLES, CLASES Y MÉTODOS IMPORTANTES</h3>

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Dropout, LSTM, Dense, Embedding
import pickle
import json
import os
import sys

num_lilith = '06'
archivo_json_original = '../../Chessia_Data_Convert/Target Data/original/Datos-V2.json'
archivo_json_expansion = '../../Chessia_Data_Convert/Target Data/expansion/Datos-V2.json'
# tokens_lilith = "../../Chessia_Data_Convert/TokensGenerator/general_tokens_v1.pkl"
tokens_lilith = "lilith03_data/tokens03.pkl"

class Partida:
    def __init__(self):
        self.movimientos = []
        self.resultado = None

# Función para cargar partidas en lotes
def cargar_partidas_por_rango(archivo_json_original, inicio=0, fin=None):
    total_movimientos = []
    total_resultados = []
    with open(archivo_json_original, 'r') as file:
        # Carga el contenido completo como una lista de objetos JSON
        lista_json = json.load(file)
        
        if fin is None or fin > len(lista_json):
            fin = len(lista_json)

        # Itera sobre cada objeto JSON en la lista
        for i in range(inicio, fin):
            total_movimientos.append(lista_json[i]['partida']['movimientos'])
            total_resultados.append(lista_json[i]['partida']['resultado'])
            
    return total_movimientos, total_resultados

def ver_tablas(history):
    fig, axs = plt.subplots(2, 1, figsize=(10, 10))

    epocas = range(1, len(history.history['loss']) + 1)

    # Graficar la precisión
    axs[0].plot(epocas, history.history['accuracy'], 'r', label='Precisión en entrenamiento')
    axs[0].plot(epocas, history.history['val_accuracy'], 'b', label='Precisión en validación')
    axs[0].set_title('Precisión en entrenamiento y validación')
    axs[0].set_xlabel('Épocas')
    axs[0].set_ylabel('Precisión')
    axs[0].legend()

    # Graficar la pérdida
    axs[1].plot(epocas, history.history['loss'], 'r', label='Pérdida en entrenamiento')
    axs[1].plot(epocas, history.history['val_loss'], 'b', label='Pérdida en validación')
    axs[1].set_title('Pérdida en entrenamiento y validación')
    axs[1].set_xlabel('Épocas')
    axs[1].set_ylabel('Pérdida')
    axs[1].legend()

    plt.tight_layout()  # Ajustar el diseño para evitar superposiciones
    plt.show()


class PlotMetricsCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        try:
            if self.model.history:
                # ver_tablas(self.model.history)
                pass

        except Exception as e:
            print("Error al mostrar la gráfica:", e)

<h3>1.0 PROCESADO DE DATOS DE LILITH - USA ProcesadorPGN_V2 Y EXTENSIÓN DE PERTIDAS</h3>

In [2]:
# Convertir el generador en una lista y extraer la primera tupla
# recogido += cargar_partidas_por_rango(archivo_json_original, fin=100000)
recogido = cargar_partidas_por_rango(archivo_json_expansion)

movimientos = recogido[0]
resultados = recogido[1]

# Crear y ajustar el tokenizador
tokenizer = Tokenizer()
with open(tokens_lilith, "rb") as f:
    tokenizer = pickle.load(f)

# Convertir todas las jugadas a secuencias de números enteros y luego a tensores válidos
movimientos_tokenized = tokenizer.texts_to_sequences(movimientos)
movimientos_tokenized = pad_sequences(movimientos_tokenized, maxlen=500, padding="pre")

resultados_tokenized = tf.one_hot([int(resultado) for resultado in resultados], depth=3)  # 3 clases: empate, victoria blanca, victoria negra

tensor_mov_entrena = tf.constant(movimientos_tokenized[:int(len(movimientos_tokenized) * 0.8) ])
tensor_res_entrena = tf.constant(resultados_tokenized [:int(len(resultados_tokenized)  * 0.8) ])
tensor_mov_prueba =  tf.constant(movimientos_tokenized[ int(len(movimientos_tokenized) * 0.8):])
tensor_res_prueba =  tf.constant(resultados_tokenized [ int(len(resultados_tokenized)  * 0.8):])

<h3>1.1 LILITH - DEFINICIÓN, ENTRENAMIENTO Y GRÁFICAS DE DATOS CON POCOS VALORES</h3>

<h4>&ensp;&ensp;1.1.1 CREACIÓN DEL MODELO</h4>

In [3]:
# Definir el modelo
modelo = tf.keras.models.load_model("lilith03_data/lilith03.keras")

# Compilar el modelo
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001) 
modelo.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

<h4>&ensp;&ensp;1.1.2 ENTRENAMIENTO</h4>

In [4]:
history = None
try:
    # Definir callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
    checkpoint = ModelCheckpoint(filepath='modelo_checkpoint.keras', save_best_only=True)
    plot_metrics = PlotMetricsCallback()

    try:
        modelo = tf.keras.models.load_model('modelo_checkpoint.keras')
        print("Checkpoint cargado. Continuando entrenamiento desde el último punto guardado...\n\n")
    except:
        print("No se encontró un checkpoint existente. Iniciando un nuevo entrenamiento...\n\n")

    # Entrenar el modelo con los callbacks
    history = modelo.fit(
        tensor_mov_entrena,
        tensor_res_entrena,
        epochs=15,
        initial_epoch=0,  # Especificar la primera época a ejecutar
        batch_size=256,
        validation_data=(tensor_mov_prueba, tensor_res_prueba),
        callbacks=[early_stopping, checkpoint, plot_metrics]
        # callbacks=[early_stopping, checkpoint]
    )

    # Guardar el modelo final
    modelo.save("lilith"+num_lilith+".keras")

    # Evaluar el modelo en los datos de prueba
    loss, accuracy = modelo.evaluate(tensor_mov_prueba, tensor_res_prueba)
    print("\n\nLoss en los datos de prueba:", loss)
    print("Precisión en los datos de prueba:", accuracy)

except:
    if history is not None:
        ver_tablas(history)
    else:
        exc_type, exc_value, exc_traceback = sys.exc_info()
        print(f"\nError: {exc_type.__name__}: {exc_value}\n")

No se encontró un checkpoint existente. Iniciando un nuevo entrenamiento...


Epoch 1/15
321/321 ━━━━━━━━━━━━━━━━━━━━ 1245s 4s/step - accuracy: 0.7449 - loss: 0.6089 - val_accuracy: 0.7553 - val_loss: 0.5822
Epoch 2/15
321/321 ━━━━━━━━━━━━━━━━━━━━ 1190s 4s/step - accuracy: 0.7721 - loss: 0.5525 - val_accuracy: 0.7650 - val_loss: 0.5675
Epoch 3/15
321/321 ━━━━━━━━━━━━━━━━━━━━ 1249s 4s/step - accuracy: 0.7882 - loss: 0.5217 - val_accuracy: 0.7686 - val_loss: 0.5589
Epoch 4/15
321/321 ━━━━━━━━━━━━━━━━━━━━ 1275s 4s/step - accuracy: 0.7961 - loss: 0.5051 - val_accuracy: 0.7692 - val_loss: 0.5600
Epoch 5/15
321/321 ━━━━━━━━━━━━━━━━━━━━ 1324s 4s/step - accuracy: 0.8068 - loss: 0.4849 - val_accuracy: 0.7716 - val_loss: 0.5567
Epoch 6/15
321/321 ━━━━━━━━━━━━━━━━━━━━ 1417s 4s/step - accuracy: 0.8122 - loss: 0.4682 - val_accuracy: 0.7696 - val_loss: 0.5620
Epoch 7/15
321/321 ━━━━━━━━━━━━━━━━━━━━ 1439s 4s/step - accuracy: 0.8157 - loss: 0.4671 - val_accuracy: 0.7700 - val_loss: 0.5618
Epoch 8/15
3

<h4>&ensp;&ensp;1.1.3 ESTADÍSTICAS Y GRÁFICAS</h4>

In [ ]:
ver_tablas(history)

<h3>1.2 PRUEBA DE PREDICCIÓN LILITH</h3>

<h4>&ensp;&ensp;1.2.1 RECOGIDA DE DATOS Y TOKENS</h4>

In [3]:
movimientos, resultados = cargar_partidas_por_rango(archivo_json_original, inicio=100000)

tokenizer = Tokenizer()
with open(tokens_lilith, "rb") as f:
    tokenizer = pickle.load(f)

tensor_mov_pred = tf.constant(pad_sequences(tokenizer.texts_to_sequences(movimientos), maxlen=500, padding="pre"))

<h4>&ensp;&ensp;1.2.2 PREDICCIÓN Y MUESTRA DE DATOS</h4>

In [4]:
np.set_printoptions(suppress=True, precision=4)

lilith = tf.keras.models.load_model("lilith"+num_lilith+".keras")

res_pred = lilith.predict(tensor_mov_pred)
aciertos = sumaAciertos = 0

for i in range(len(res_pred)):
    if int(resultados[i]) != int(np.argmax(res_pred[i])): char = '❌'
    else: 
        char = '✅'
        aciertos += 1
        sumaAciertos += res_pred[i][np.argmax(res_pred[i])]

    print(f"Predicción: {str(res_pred[i]):>22} -- Dice: {np.argmax(res_pred[i])} -- Real: {resultados[i]}  {char}  --- Aciertos: {aciertos:>5}/{i+1:<5} - {int(aciertos/(i+1)*100):>3}%")


105/105 ━━━━━━━━━━━━━━━━━━━━ 54s 512ms/step
Predicción: [0.9563 0.019  0.0246] -- Dice: 0 -- Real: 0  ✅  --- Aciertos:     1/1     - 100%
Predicción: [0.8785 0.1157 0.0058] -- Dice: 0 -- Real: 1  ❌  --- Aciertos:     1/2     -  50%
Predicción: [0.7773 0.137  0.0857] -- Dice: 0 -- Real: 1  ❌  --- Aciertos:     1/3     -  33%
Predicción: [0.1008 0.8883 0.0109] -- Dice: 1 -- Real: 1  ✅  --- Aciertos:     2/4     -  50%
Predicción: [0.0486 0.9499 0.0015] -- Dice: 1 -- Real: 1  ✅  --- Aciertos:     3/5     -  60%
Predicción: [0.0229 0.9745 0.0026] -- Dice: 1 -- Real: 1  ✅  --- Aciertos:     4/6     -  66%
Predicción: [0.5548 0.4368 0.0083] -- Dice: 0 -- Real: 1  ❌  --- Aciertos:     4/7     -  57%
Predicción: [0.1069 0.0314 0.8617] -- Dice: 2 -- Real: 2  ✅  --- Aciertos:     5/8     -  62%
Predicción: [0.6268 0.3553 0.0179] -- Dice: 0 -- Real: 0  ✅  --- Aciertos:     6/9     -  66%
Predicción: [0.1984 0.0816 0.72  ] -- Dice: 2 -- Real: 0  ❌  --- Aciertos:     6/10    -  60%
Predicción: [0.3

In [5]:
print(f"{sumaAciertos/len(res_pred)}")

0.6784049469351733
